# SBUS Demo Notebook

This notebook demonstrates the SBUS workflow using the organized data structure.

**Links:**
- [Source data](https://ichoosr.egnyte.com/navigate/folder/701cce7f-a797-47c6-bcae-7ee2e3242e8c)
- [Confluence documentation](https://ichoosr.atlassian.net/wiki/x/IYBwMQE?atlOrigin=eyJpIjoiMzg2YWQxZmI2NjUyNDg1Y2E2YjMyZTViOGRjOWFmOGEiLCJwIjoiYyJ9)

In [ ]:
# Setup: Load IPython extensions and configure environment
%reload_ext setup

In [ ]:
# Optional: Test ZhipuAI API connection directly
import os
from igent.connectors.endpoints import EndpointsChatCompletionClient
from autogen_core.models import ModelInfo, UserMessage

# Get API key from environment (recommended) or set directly
API_KEY = os.getenv("ZAI_API_KEY", "4c8c8f6215624cbc8e9e045bff9de8e0.dIXYlpSJWll1wWhy")

# Create a test client
model_client = EndpointsChatCompletionClient(
    endpoint="https://api.z.ai/api/paas/v4/",
    api_key=API_KEY,
    model="glm-4.5-air",
    model_info=ModelInfo(
        vision=False,
        function_calling=True,
        json_output=False,
        family="gpt-4o",
    ),
)

# Test the connection
result = await model_client.create([UserMessage(content="Hello, world!", source="user")])
print(f"✓ API connection successful: {result.content}")

## Scenario-Based Workflow (Recommended)

Use scenario configuration files for clean, organized testing. Each scenario defines:
- Which registration dataset to use
- Which offers file to use
- Output file paths

**Available scenarios:**
- `overlap_only.yaml` - High competition (all registrations in overlapping ZIPs)
- `full_dataset.yaml` - Diverse distribution (41 different ZIP codes)
- `no_battery.yaml` - Product constraint (All Energy Solar missing batteries)

In [ ]:
# Reset capacity before running (important!)
!python ../scripts/reset_capacity.py --all

In [ ]:
# Run workflow with a scenario configuration
from igent.workflows.p1m1m2c import run_workflow_from_scenario

await run_workflow_from_scenario(
    scenario_file="../data/sbus/scenarios/overlap_only.yaml",
    model="zai_glm4_5_air",  # Options: zai_glm4_5_air, zai_glm4_6, openai_gpt4o, openai_gpt5, azure
    max_items=5,
    stream=False
)

## Direct File Path Workflow (Legacy)

For backward compatibility, you can still specify files directly without using scenarios.

In [ ]:
# Direct file path approach (less recommended)
from igent.workflows.p1m1m2c import run_workflow

await run_workflow(
    model="zai_glm4_5_air",
    stream=False,
    business_line="sbus",
    registrations_file="../data/sbus/registrations/overlap_only.json",
    offers_file="../data/sbus/offers/base_offers.json",
    matches_file="../data/sbus/results/matches.json",
    pos_file="../data/sbus/results/pos.json",
    stats_file="../data/sbus/results/stats.csv",
    max_items=5
)

## List Available Scenarios

Use this to see what scenario configurations are available.

In [ ]:
# List all available scenarios
from igent.utils import list_scenarios, load_scenario

scenarios = list_scenarios("../data/sbus/scenarios")
print("Available scenarios:")
for scenario_file in scenarios:
    config = load_scenario(scenario_file)
    print(f"\n  📋 {scenario_file}")
    print(f"     {config['name']}")
    print(f"     {config['description']}")